# Linear Algebra and Optimisation

###### COMP4670/8600 - Introduction to Statistical Machine Learning - Tutorial 1B

$\newcommand{\trace}[1]{\operatorname{tr}\left\{#1\right\}}$
$\newcommand{\Norm}[1]{\lVert#1\rVert}$
$\newcommand{\RR}{\mathbb{R}}$
$\newcommand{\inner}[2]{\langle #1, #2 \rangle}$
$\newcommand{\DD}{\mathscr{D}}$
$\newcommand{\grad}[1]{\operatorname{grad}#1}$
$\DeclareMathOperator*{\argmin}{arg\,min}$

Setting up python environment ([do not use pylab](http://carreau.github.io/posts/10-No-PyLab-Thanks.ipynb.html))

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import scipy.optimize as opt
import time

%matplotlib inline

Consider the following cost function $ f(X) $ defined
over the space of real $ n \times p $ matrices
\begin{equation}
  f(X) = \frac{1}{2} \trace{X^T C X N} + \mu \frac{1}{4} \Norm{N - X^T X}^2_F
\end{equation}
where $ X \in \RR^{n \times p} $, $ n \ge p $, and the matrix $ C $ is symmetric, 
such that $ C = C^T $. The scalar $ \mu $ is assumed to be larger than the $p$th smallest 
eigenvalue of $ C $. The matrix $ N $ is diagonal with distinct positive entries
on the diagonal.
The trace of a square matrix $ A $ is denoted as $ \trace{A} $, and
the Frobenius norm of an arbitrary matrix $ A $ is defined as $ \Norm{A}_F = \sqrt{\trace{A^T A}} $.



## Frobenious Norm

Implement a Python function ```frobenius_norm``` which accepts an arbitrary matrix $ A $ and returns
$ \Norm{A}_F $ using the formula given. (Use ```numpy.trace``` and ```numpy.sqrt```.)
1. Given a matrix $ A \in \RR^{n \times p} $, what is the complexity of your implementation of ```frobenius_norm```
using the formula above?
2. Can you come up with a faster implementation, if you were additionally told that $ p \ge n $ ?
3. Can you find an even faster implementation than in 1. and 2.? 

### Solution description

In [1]:
import numpy
def frobenius_norm(A):
    """ Accepts Matrix A and returns the Forbenius Norm of A
        
        (matrix) -> (number)
    """
    return numpy.sqrt(numpy.trace(numpy.transpose(A) * A))

def frobenius_norm_efficient(A):
    """ 
    """
    n = A.shape[0]
    p = A.shape[1]
    #return numpy.dot(numpy.vec(numpy.transpose(A)), numpy.vec(A))
    return numpy.sqrt(sum([A[i,j]**2 for i in range(n) for j in range(p)]))

print (frobenius_norm(numpy.mat([[1,2,3],[4,5,6],[7,8,9],[9,5,3]])))
print (frobenius_norm_efficient(numpy.mat([[1,2,3],[4,5,6],[7,8,9],[9,5,3]])))

20.0
20.0


## Cost Function $f(X)$ with matrix argument

Implement the cost function defined as $f(X)$ above as a function ```cost_function_for_matrix```
in Python.

Hint: As good programmers, we do not use global variables in subroutines.


In [3]:
def cost_function_for_matrix(X, mu):
    C = construct_random_matrix()
    N = construct_diagonal_matrix()
    cost = 0.5 * numpy.trace(numpy.transpose(X) * C * X * N) + \
        mu * 0.25 * (frobenius_norm_efficient(N - numpy.transpose(X) * X) ** 2)
    return cost

## Cost Function $f(X)$ with vector argument

Many standard optimisation routines work only with vectors. Fortunately, as vector spaces,
the space of matrices $ \RR^{n \times p} $ 
and the space of vectors $ \RR^{n p} $ are isomorphic. What does this mean?

Implement the cost function $ f(X) $ given $ X $ as a vector and call it ```cost_function_for_vector```.
Which extra arguments do you need for the cost function?


In [12]:
def cost_function_for_vector(x, mu):
    x = numpy.mat(x).T
    C = random_matrix_from_eigenvalues(numpy.random.randint(10, size=numpy.shape(x)[0]))
    N = 10
    cost = 0.5 * numpy.trace(x.T * C * x * N) + \
        mu * 0.25 * (N - numpy.dot(x.T,x))
    return cost

## Construction of a random matrix $C$ with given eigenvalues

A diagonal matrix has the nice property that the eigenvalues can be directly read off
the diagonal. Given a diagonal matrix $ C \in \RR^{n \times n} $ with distinct eigenvalues, 
how many different diagonal matrices have the same set of eigenvalues?

Ans: $n!$

Given a diagonal matrix $ C \in \RR^{n \times n} $ with distinct eigenvalues,
how many different matrices have the same set of eigenvalues?

Given a set of $ n $ distinct real eigenvalues $ \mathcal{E} = \{e_1, \dots, e_n \} $, 
write a Python function ```random_matrix_from_eigenvalues``` which takes a list of
eigenvalues $ E $ and returns a random symmetric matrix $ C $ having the same eigenvalues.

### Solution description

In [5]:
def random_matrix_from_eigenvalues(E: list) -> numpy.matrix:
    rand_matrix = numpy.random.random_integers(1,20, (len(E), len(E)))
    symm_matrix = rand_matrix + rand_matrix.T
    diag_matrix = numpy.mat(numpy.diag(E))
    return symm_matrix * diag_matrix * symm_matrix.T

print (random_matrix_from_eigenvalues(numpy.random.randint(10, size=10)))

[[43935 30956 29216 38050 37672 38081 28552 32337 31594 40431]
 [30956 26749 24128 28901 28474 29010 21368 23713 25134 31012]
 [29216 24128 25118 28218 28501 27375 23169 23304 19514 29699]
 [38050 28901 28218 42533 37690 36868 29692 36251 28310 39674]
 [37672 28474 28501 37690 37225 35095 29021 31868 26525 38537]
 [38081 29010 27375 36868 35095 35720 27145 31495 28949 37904]
 [28552 21368 23169 29692 29021 27145 24568 25062 17423 29703]
 [32337 23713 23304 36251 31868 31495 25062 31528 23802 33623]
 [31594 25134 19514 28310 26525 28949 17423 23802 31566 30605]
 [40431 31012 29699 39674 38537 37904 29703 33623 30605 41196]]


## Minimising the cost function $f(X)$

Use the minimisation functions ```fmin``` or ```fmin_powell``` provided in the
Python package ```scipy.optimize``` to minimise the cost function ```cost_function_for_vector```.

Hint: Use the argument ```args``` in the minimisation functions  ```fmin``` or ```fmin_powell``` 
to provide the extra parameters to
your cost function ```cost_function_for_vector```.


In [13]:
from scipy.optimize import fmin, fmin_powell
x = numpy.mat(numpy.random.random_integers(1,20,10)).T
#opt.fmin(cost_function_for_vector, x, (1,))
fmin_powell(cost_function_for_vector, x, (1,))

Optimization terminated successfully.
         Current function value: 159121464.620811
         Iterations: 3
         Function evaluations: 451


matrix([[-50.64287727,  17.38196597,  -0.19701183,  80.94875723,
           7.86485133,  25.36278399,  -9.92901549, -17.63793053,
         -88.28335912, -19.36043994]])

## Gradient of $f(X)$

Calculate the gradient for the cost function $f(X)$ given the
inner product on the space of real matrices $ n \times p $ is defined as
\begin{equation}
  \inner{A}{B} = \trace{A^T B}
\end{equation}

Implement a function ```gradient_for_vector``` which takes $ X $ as a vector, and
returns the gradient of $ f(X) $ as a vector.


### Solution description

$$D(f(X), \eta) = \lim_{\epsilon \rightarrow 0} \Big[\frac{f(X + \epsilon \eta) - f(X)}{\epsilon} \Big]$$
$f(X) = \frac{1}{2} \trace{X^T C X N} + \mu \frac{1}{4} \Norm{N - X^T X}^2_F$

$f(X + \epsilon \eta) = \frac{1}{2} \trace{(X + \epsilon \eta)^T C (X + \epsilon \eta) N} + \mu \frac{1}{4} \Norm{N - (X + \epsilon \eta)^T (X + \epsilon \eta)}^2_F$

$$\lim_{\epsilon \rightarrow 0} \Big[\frac{f(X + \epsilon \eta) - f(X)}{\epsilon} \Big] = \frac{1}{2} \trace{X^T C \eta N + \eta^T CXN}$$

$$\Rightarrow D(f(X), \eta) = \frac{1}{2} \big[\trace{X^T C \eta N} + \trace{\eta^T CXN}\big]$$

$= \frac{1}{2} \big[\trace{X^T C \eta N} + \trace{N^T X^T C^T \eta}\big] \tag{$\trace{A^T} = \trace{A}$}$

$= \frac{1}{2} \big[\trace{X^T C \eta N} + \trace{NX^T C \eta}\big]\tag{$N$ is Diagonal, $C$ is symmetric}$

$= \frac{1}{2} \big[\trace{X^T C \eta N} + \trace{NX^T C \eta}\big]\tag{$N$ is Diagonal, $C$ is symmetric}$

## Minimising the cost function $f(X)$ using the gradient

Use the minimisation functions ```fmin_cg``` or ```fmin_bfgs``` provided in the
Python package ```scipy.optimize``` to minimise the cost function ```cost_function_for_vector``` utilising the gradient ```gradient_for_vector```.

Compare the speed of convergence to the minimisation with ```fmin``` or ```fmin_powell```.


In [7]:
# Solution goes here

## Minima of $f(X)$

Compare the columns $x_1,\dots, x_p$ of the matrix $X^\star$ which minimises $ f(X) $ 
\begin{equation}
  X^\star = \argmin_{X \in \RR^{n \times p}} f(X)
\end{equation}

with the eigenvectors related to the smallest eigenvalues of $ C $.


### Solution description

In [8]:
# Solution goes here